In [3]:
# 这里保留文件所处文件夹的路径，路径中包含文件本身名称，譬如"/Users/jack/Documents/fs_project/aaa.docx"
import os
import openpyxl

# 设置要遍历的根目录
root_dir = "/Users/jack/Documents/xx"

# 创建一个新的 Excel 工作簿
workbook = openpyxl.Workbook()
worksheet = workbook.active

# 定义一个递归函数来遍历文件夹和文件
def traverse_directory(dir_path, row):
    for item in os.listdir(dir_path):
        item_path = os.path.join(dir_path, item)
        if os.path.isdir(item_path):
            # 如果是文件夹,递归遍历子文件夹
            row = traverse_directory(item_path, row)
        else:
            # 如果是文件,将文件名和绝对路径写入表格
            worksheet.cell(row=row, column=1, value=item)
            worksheet.cell(row=row, column=2, value=item_path)
            row += 1
    return row

# 开始遍历根目录
traverse_directory(root_dir, 1)

# 保存 Excel 文件
workbook.save("/Users/jack/Documents/xx.xlsx")
print("文件已保存到当前目录下的 XC_filelist.xlsx 中。")

文件已保存到当前目录下的 XC_filelist.xlsx 中。


In [4]:
# 这里只保留文件所处文件夹的路径，去除具体文件名称

import os
import re
import openpyxl

# 读取 Excel 文件
excel_file = "/Users/jack/Documents/xx.xlsx"
workbook = openpyxl.load_workbook(excel_file)
worksheet = workbook.active

# 定义正则表达式模式
pattern = r"/Users/chengyong/Documents/folder_name((?:.*?/){2,})"

# 遍历 B 列并替换路径
for row in range(2, worksheet.max_row + 1):
    cell = worksheet.cell(row=row, column=2)  # B 列
    full_path = cell.value
    if full_path:
        relative_path = re.search(pattern, full_path)
        if relative_path:
            cell.value = relative_path.group(1)[:-1]  # 去掉最后一个斜杠

# 保存 Excel 文件
workbook.save(excel_file)
print("文件已更新并保存。")

文件已更新并保存。


In [ ]:
# 第三步，在 Excel 中操作，对 A 列的路径排序；
# 第四步，复制一下表格到 sheet2，把文件名放在 A 列，把路径放在 B 列，复制B 列到 C 列，对 C 列进行分列操作，把二级目录，三级目录摘出来。

下面的程序是利用正则表达式，批量读取文件名中的日期，并写入到另外一列的功能。
注意：这里应用了\d 和重复次数来提取至少 6 位及以上连续的数字。
- r'\d+'寻找任何数字的正则表达式,'+'的含义见正则表达式 repetted numbers,意思是返回整个组合
- r'\d'会将数字拆开，一个一个返回。
- r'\d{6,}'排除了譬如文件中 “01”这样的两位连续数字，

In [12]:
import re
# r'\d+'寻找任何数字的正则表达式,'+'的含义见正则表达式 repetted numbers,意思是返回整个组合
# r'\d'会将数字拆开，一个一个返回。
# r'\d{6,}'排除了譬如文件中 “01”这样的两位连续数字， 
def findate(filepath):
    pattern = re.compile(r'\d{6,}')
    word = re.findall(pattern, filepath) # 定义函数，别忘了返回一个数值
    if word:
        return word
    else:
        return 'none'
    


In [23]:
# test 函数健壮性，会不会误识别出“01”或者“2024（1）号”
findate("01 沪 张三股份2024（1）号  20240510 makup.docx")

['20240510']

In [ ]:
import csv

words = [] # 注意要先声明一个空的列表，将 csv 的列都存在一个 list 中

# 这里引入 csv 具体路径，我约定的是和程序同一文件夹下
with open('zx.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        words.append(row[0])
print(words)
        

In [14]:
# 写入时候，如果 CSV 不存在，程序会自动创建一个新的文件写入
with open('zx_1.csv', 'w', newline='' ) as file:
    writer = csv.writer(file)
    writer.writerow(['filepath','date'])
    
    for word in words:
        date = findate(word)
        if date != 'none': # 判断语句很重要，不然，如果没有日期，直接 writerow，会错位
            writer.writerow([word, date[0]]) # 这里date是一个list，元素只有1个，要写入具体日期，就要用 date[0]访问
        else:
            writer.writerow([word, 'none'])